In [1]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output

In [2]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)
print("done!")

done!


In [3]:


# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)


#X_train, X_valid, X_angle_train, X_angle_valid, y_train, y_valid = train_test_split(X_train
#                    , X_angle_train, y_train, random_state=123, train_size=0.75)



In [45]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    
def get_model():
    bn_model = 0
    p_activation = "elu"
    p = 0.2
    p_dense = 0.6
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(16,3,3, activation=p_activation) (input_1)
    img_1 = Conv2D(16,3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(p)(img_1)
    img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
    img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(p)(img_1)
    img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
    img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(p)(img_1)
    img_1 = Conv2D(128, 3,3, activation=p_activation) (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(p)(img_1)
    img_1 = GlobalMaxPooling2D() (img_1)
    
    
    img_2 = Conv2D(128, 3,3, activation=p_activation) ((BatchNormalization(momentum=bn_model))(input_1))
    img_2 = MaxPooling2D((2,2)) (img_2)
    img_2 = Dropout(p)(img_2)
    img_2 = GlobalMaxPooling2D() (img_2)
    
    img_3 = BatchNormalization(momentum=bn_model)(input_2)
    
    img_concat =  (Concatenate()([img_1, img_2, img_3]))
    
    dense_ayer = Dropout(p_dense) (BatchNormalization(momentum=bn_model) ( Dense(256, activation=p_activation)(img_concat) ))
    dense_ayer = Dropout(p_dense) (BatchNormalization(momentum=bn_model) ( Dense(64, activation=p_activation)(dense_ayer) ))
    output = Dense(1, activation="sigmoid")(dense_ayer)
    
    model = Model([input_1,input_2],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model
model = get_model()
model.summary()

/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
X_1 (InputLayer)                 (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
conv2d_191 (Conv2D)              (None, 74, 74, 16)    208         X_1[0][0]                        
____________________________________________________________________________________________________
conv2d_192 (Conv2D)              (None, 73, 73, 16)    1040        conv2d_191[0][0]                 
____________________________________________________________________________________________________
max_pooling2d_124 (MaxPooling2D) (None, 36, 36, 16)    0           conv2d_192[0][0]                 
___________________________________________________________________________________________

In [46]:
file_path = "/mnt/extDisk/courses/data/statoil/data/train/.model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model = get_model()
model.fit([X_train, X_angle_train], y_train, epochs=25
          , validation_data=([X_valid, X_angle_valid], y_valid)
         , batch_size=32
         , callbacks=callbacks)



/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), activation="elu")`
/home/user/tensorflow/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 

Train on 602 samples, validate on 200 samples
Epoch 1/25
602/602 [==============================] - 5s - loss: 0.9727 - acc: 0.6013 - val_loss: 0.8972 - val_acc: 0.6550
Epoch 2/25
602/602 [==============================] - 0s - loss: 0.8556 - acc: 0.5980 - val_loss: 1.1202 - val_acc: 0.6400
Epoch 3/25
602/602 [==============================] - 0s - loss: 0.7551 - acc: 0.6545 - val_loss: 1.8569 - val_acc: 0.6900
Epoch 4/25
602/602 [==============================] - 0s - loss: 0.8086 - acc: 0.6296 - val_loss: 2.1616 - val_acc: 0.5900
Epoch 5/25
602/602 [==============================] - 0s - loss: 0.7580 - acc: 0.6528 - val_loss: 1.7221 - val_acc: 0.6000
Epoch 6/25
602/602 [==============================] - 0s - loss: 0.7193 - acc: 0.6478 - val_loss: 1.9411 - val_acc: 0.6150
Epoch 7/25
602/602 [==============================] - 0s - loss: 0.6739 - acc: 0.6827 - val_loss: 2.1364 - val_acc: 0.6600


In [47]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
600/602 [============================>.] - ETA: 0s[0.90748327495251779, 0.67441859831445639]
####################
watch list evaluate:
200/200 [==============================] - 0s
[0.89721262454986572, 0.65499997138977051]


In [6]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8424/8424 [==============================] - 1s     


In [7]:
prediction = np.clip(prediction,0.05,0.95)

In [8]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.084889
1,4023181e,0.074619
2,b20200e4,0.050000
3,e7f018bb,0.950000
4,4371c8c3,0.093100
5,a8d9b1fd,0.288849
6,29e7727e,0.314339
7,92a51ffb,0.950000
8,c769ac97,0.091143
9,aee0547d,0.050000


In [9]:
submission.to_csv("/mnt/extDisk/courses/data/statoil/data/train/keras_baseline_clip.csv", index=False)